<a href="https://colab.research.google.com/github/zerotodeeplearning/ztdl-masterclasses/blob/master/solutions_do_not_open/Model_Serving_with_Flask_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Catalit LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Serving with Flask

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization

import os
import json
import shutil
import gzip
import dill
import requests

### Scikit Learn model serving with Flask

In [ ]:
url = "https://raw.githubusercontent.com/zerotodeeplearning/ztdl-masterclasses/master/data/"

In [ ]:
df = pd.read_csv(url + 'wikipedia_languages.csv')
classes = df['language'].unique()

X = df['sentence']
y = df['language']

### Exercise 1

Build and train simple model using Scikit-Learn that classifies the sentences into their respective languages. For this exercise you can avoid doing train/test split.

In [ ]:
tfidf_vect = TfidfVectorizer(ngram_range=(1, 3),
                             analyzer='char',
                             max_features=5000)
estimator = LogisticRegression(solver='liblinear', C=10)

model = make_pipeline(tfidf_vect, estimator)

model.fit(X, y)

#### Model saving and sanity check:

In [ ]:
with gzip.open('sklearn_model.dill.gz', 'wb') as f:
    dill.dump(model, f)

In [ ]:
test_sentences = ['this is a sentence in english',
                  'questa è una frase in Italiano']

In [ ]:
model.predict(test_sentences)

#### Flask serving

In [ ]:
%%writefile flask_sklearn.py
import dill
import gzip
import json
import numpy as np
from flask import Flask
from flask import request, jsonify


app = Flask(__name__)

# Load model
with gzip.open('sklearn_model.dill.gz') as fin:
  loaded_model = dill.load(fin)
print("Model loaded in memory. Ready to roll!")


# convert json to array
def preprocess(data):
  res = json.loads(data)
  return np.array(res['data'])


# generate predictions
@app.route('/', methods=["POST"])
def predict():
  if request.method == "POST":
    data = request.data
    print("Data:", data)

    processed = preprocess(data)
    print("Processed:", processed)

    predictions = loaded_model.predict(processed)
    print("Predictions:", predictions)

    return jsonify(predictions.tolist())

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

In [ ]:
!pkill -f flask_sklearn.py
!pkill -f flask_keras.py

In [ ]:
get_ipython().system_raw(
    'python3 flask_sklearn.py > sklearn_logs.txt 2>&1 &'
)

In [ ]:
!tail sklearn_logs.txt

#### Predictions with Curl

In [ ]:
!curl -d '{"data": ["this is a sentence in english", "questa è una frase in Italiano"]}' \
     -H "Content-Type: application/json" \
     -X POST http://localhost:5000

#### Predictions with Python requests

In [ ]:
api_url = "http://localhost:5000/"
payload = {'data': test_sentences}
headers = {'content-type': 'application/json'}

response = requests.post(api_url,
                         data=json.dumps(payload),
                         headers=headers)

In [ ]:
response.json()

### Tensorflow model serving with Flask

In [ ]:
df = pd.read_csv(url + 'wifi_location.csv')
df.head()

In [ ]:
X = df.drop('location', axis=1).values
y = df['location'].values

### Exercise 2

Build and train simple model using Tensorflow and Keras that classifies the Wi-Fi data into their respective rooms. For this exercise you can avoid doing train/test split.

In [ ]:
inputs = Input(shape=(7,))
x = BatchNormalization()(inputs)
x = Dense(50, activation='relu')(x)
x = Dense(30, activation='relu')(x)
x = Dense(10, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

model.compile('adam',
              'sparse_categorical_crossentropy',
              metrics=['accuracy'])

h = model.fit(X, y,
              batch_size=128,
              epochs=40,
              verbose=0,
              validation_split=0.1)

pd.DataFrame(h.history).plot()
plt.ylim(0, 1);

#### Model saving and sanity check:

In [ ]:
model.save('keras_model')

In [ ]:
test_indexes = [0, 550, 1032]

In [ ]:
y[test_dat]

In [ ]:
test_data = X[test_indexes].tolist()
test_labels = y[test_indexes].tolist()
print("Test data:", test_data)
print("Test labels:", test_labels)

In [ ]:
model.predict(test_data).argmax(axis=1)

### Exercise 3: Model serving with Flask

Modify the scikit learn script to work with tensorflow:

- adapt the imports
- replace the model loading part using `tf.keras.models.load_model`
- adapt the prediction part so that it returns the room number

Your script should look like:
```python
%%writefile flask_keras.py
# YOUR CODE HERE


app = Flask(__name__)

# Load model

# YOUR CODE HERE

print("Model loaded in memory. Ready to roll!")


# convert json to array
def preprocess(data):
  res = json.loads(data)
  return np.array(res['data'])


# generate predictions
@app.route('/', methods=["POST"])
def predict():
  if request.method == "POST":
    data = request.data
    print("Data:", data)

    processed = preprocess(data)
    print("Processed:", processed)

    # YOUR CODE HERE
    print("Predictions:", predictions)

    return jsonify(predictions.tolist())

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)
```


- Run the script using the code above
- Check in the logs that it's running correctly
- Send some test data to the flask server using either `curl` or `requests`

In [ ]:
%%writefile flask_keras.py
import json
import numpy as np
from flask import Flask
from flask import request, jsonify
import tensorflow as tf


app = Flask(__name__)

# Load model
loaded_model = tf.keras.models.load_model('keras_model')
print("Model loaded in memory. Ready to roll!")


# convert json to array
def preprocess(data):
  res = json.loads(data)
  return np.array(res['data'])


# generate predictions
@app.route('/', methods=["POST"])
def predict():
  if request.method == "POST":
    data = request.data
    print("Data:", data)

    processed = preprocess(data)
    print("Processed:", processed)

    probabilities = loaded_model.predict(processed)
    predictions = probabilities.argmax(axis=1)
    print("Predictions:", predictions)

    return jsonify(predictions.tolist())

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

In [ ]:
!pkill -f flask_sklearn.py
!pkill -f flask_keras.py

In [ ]:
get_ipython().system_raw(
    'python3 flask_keras.py > keras_logs.txt 2>&1 &'
)

In [ ]:
!tail keras_logs.txt

In [ ]:
!curl -d '{{"data": {test_data} }}' \
     -H "Content-Type: application/json" \
     -X POST http://localhost:5000

In [ ]:
api_url = "http://localhost:5000/"
payload = {'data': test_data}
headers = {'content-type': 'application/json'}

response = requests.post(api_url,
                         data=json.dumps(payload),
                         headers=headers)
response.json()